# Correlations
Here we present the study of the relationships between the key features we've 
picked out.

#### Environment Setup

In [ ]:
## external
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import autocorrelation_plot

## stdlib
from pathlib import Path
from typing import Any, Callable
from pprint import pprint

#### Data Loading & Setup